In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import json
import gdown

# pd.set_option('display.max_rows', None)
# pd.set_option('mode.chained_assignment',None)

In [2]:
gdown.download(id = '1h0HiImmEonZCBUcRXOwYAi9g39nYs-RX')

Downloading...
From: https://drive.google.com/uc?id=1h0HiImmEonZCBUcRXOwYAi9g39nYs-RX
To: /content/georef-australia-state-suburb.csv
100%|██████████| 53.5M/53.5M [00:00<00:00, 77.4MB/s]


'georef-australia-state-suburb.csv'

In [3]:
df = pd.read_csv('/content/georef-australia-state-suburb.csv',sep=';')
del df['Geo Shape'], df['Geo Point'], df['Type'], df['Year'], df['Iso 3166-3 Area Code']

* The website had only sugestions to pull data from.
* so we have a sugestion genrator with 3 charecters.
* its total combinations lengeth were 17576

In [14]:
from itertools import product
from string import ascii_lowercase
keywords = tuple(map(''.join, product(ascii_lowercase, repeat=3)))
print(len(keywords))

17576


In [15]:
area_list = df['Official Name Local Government Area']

In [16]:
headers = {
    'authority': 'www.domain.com.au',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/json',
    # 'cookie': 'domain-mixpanel-id_ab0bde70050c3eabaaf8824402fa01e0=$device:188995b514457e-03d062656d4fe5-26031d51-e0985-188995b5145100b; DEVICE_SESSIONID=a023599d-f7d9-46eb-b321-f121e875624e; searchSOI=nsw; bm_sz=E5C3168193AD41FE541960D95436255D~YAAQX+hUuDpcuJaIAQAA3jpJrhTax8oHVn/x4xpTKyn+pZ3tBypaiFN+fhR/FkbUiUtsbtuptrKHlrQVtLONesMfl+u22kTH6Oe8fVzvoXh0i9/cktskT2ksu8Pl/sK2ke2twQKuLm7o72/j1Zlmj/NztVuvVJHbLx2PWFbJJSVaJroFVp94sNdqGqmQI4m79RXMWA2cJHdE/EMHzxxRm3eeZ0wK365YXrDIM1MggRFMgn8svljDLLWJWCpLyPWBw7cbNNfG4viqhJZSnPCtngE5yd2wCks5OL3m4UX0p2YiWKhVvjefSvZJSq/wRTYWAiQwvgnyyp5HNofRq20=~3421252~4473143; bm_mi=D952F10DCA95880574A8BC722F2EB220~YAAQX+hUuCPXuJaIAQAA0bBarhSxMOjumckKnKAlJGqyac9HJv7nNQQY018qrEOD6li5mlKJJdlDNNnQMNimYXRtxDPmKz5LEZvmj16ofM4zzp1LEoVU9F6EYvK6d4Ip0JJi5ZveAW787x1wVwMpepTQdUBDRpiPzNBKDnV0+8dI12TDQIIgIetF7fMeiQoqfEon4lKfpkhnH1TxQtk4qj2a/LWB2L3d4lue7GxH6gjdtlP2fM1aZcux9RSVPu1DVT/kxVCwUYnQ/rqfYsszfYcWGD+MbOI6nSKiHeuMXKb70P8IDPt9MNraiAvYW9bte2h28jNHksjvzb1xXKvr~1; ak_bmsc=1DE2C239D9A96F1142226B5FA5500183~000000000000000000000000000000~YAAQX+hUuBnYuJaIAQAAcNxarhTQh3sbnvdeNNlrfoGWlFkM3eHfk14mHOm1PZyIU0gmYFxPO9/qdHza1NrXcHuNpWrNwVvbg1dISowLTtF7HsRvUEX7jU+wau+gQjX6thKlHYUF3K50ITyCdJE8dscqRM5NoYZj7uO0M/LIZpD6beritGyU896dEKD/2dPy/o7CGykFe56byEq8XJv3tFoJyQb1fIW8twUkzznRLOKajOXSd/klW/N6rLuvXGJjoFFfWEvBgi4VI4eMtUuGMiQZRQohNzTQ8RbVyI6r+VvPpSIlTqirB6rVBlk2MpvrWdIG6luUwoGnR2ddw6TIAvxcRK2tt3aNdHT84ggGEOn6A+UM+KndYFpWCTHs20VPTUkN2e44Zx5xqTS3zSGMbqvfWarNWtaOtm40CUxLtNdfXgqWJCEytUPJPM0/; bm_sv=A248E62E876DC5FACA1C1B94D0727478~YAAQX+hUuIa9uZaIAQAARvZ7rhQmRelJL4ghZ5opdMFxpF0zmhQLqwSZ2hKhHDu+nS4kBVQLt+JtTF4BO2IMCRWdw76Zsbhp6siOtsux3Rx/s71QUW1TztDbZE/4UyxlV02YXRDKxI2QwV8vQOa9FdGp43thdH3hsqngJNLgI4qBF7/whqCTKRIM23c7wzHh6FMDDWanOVq+Olkac3fGqtE0bH3uK9OxwL5iw7WT6qIbRb9vtC1odx86FasDcBAT5Inf8w==~1; _abck=1DD95BDDF3FCA5D055B5FB003FEBDFF4~0~YAAQX+hUuGW+uZaIAQAAPv97rgokC/jffin+CGwE6ip6uInGb+3nw3Eibix6Nd7g4m/9XzeDvpAJRj3jl30Hp2K6KBS9nLDQrwOx0+O1uxtdrvEu0LRI+e/paHjKnl/Rtmqh2JpWCqTtljFKgjFFr7mVhgDxcSwvuIJZ6YsSOZ05ofNiw9yfkHF0JcAIYYUSNTqgD/GG/RKkcW7xrE1LAxJutk06NJNXOo0WfGpnI+c3Kx6P5YU7nBvve+hi7aeCAwBJY5TSOypVZtDAdCrbUmbzYQjeJcL/DAr6VEGFeL8DOee3iIhi/X7wGnk/gl9cJsOrEQfx54sYX5JlPBMef7GBkKsS/xTbR5HAXHi+hj/OdKa+cwfslumdx7yqTtYKdtqb6jtWQWqQBtQE~-1~-1~1686552014',
    'origin': 'https://www.domain.com.au',
    'referer': 'https://www.domain.com.au/suburb-profile/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Brave";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

In [17]:
def extract_first_name(name):
    # Remove special characters using regex
    cleaned_name = re.sub('[^a-zA-Z ]', '', name)
    # Extract the first name
    first_name = cleaned_name.split()[0]
    return first_name

def extract_first_names(names):
    # Split the names by comma
    name_list = names.split(',')
    # Extract the first name from each name
    first_names = [extract_first_name(name) for name in name_list]
    return first_names

In [20]:
unique_names = set()
for sub_urb in keywords:
  sub_urb = extract_first_names(sub_urb)
  try:
    json_data = {
        'variables': {
            'input': sub_urb[0],
            'maxResults': 2000,
            'locationTypes': [
                'SUBURB',
            ],
        },
        'query': '\n        query SuggestLocations(\n          $input: String!\n          $locationTypes: [LocationSuggestionType!]\n          $maxResults: Int\n        ) {\n          suggestLocations(\n            prefixText: $input,\n            locationTypes: $locationTypes,\n            maxResults: $maxResults,\n            exactMatchesToTop: true) {\n            __typename\n            ... on SuburbSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n            ... on AreaSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n            ... on RegionSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n            ... on SchoolSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n          }\n         }\n       ',
    }

    response = requests.post('https://www.domain.com.au/graphql', headers=headers, json=json_data)

    if response.status_code == 200:
        data = response.json()
        length_of_data= len(data['data']['suggestLocations'])
        for i in range(length_of_data):
          value = data['data']['suggestLocations'][i]['value']
          unique_names.add(value)
  except:   
    None     


* trying to collect the sugestions we have collected 15879 suburbs instard of 15303

In [23]:
print(len(unique_names))

15879


In [22]:
df = pd.DataFrame(unique_names)
df.to_csv('urls.csv') 

In [ ]:
my_dict = response.json()
df_newlyReleased = pd.DataFrame(my_dict['newlyReleased'])
df_listingCountsByState = pd.DataFrame(my_dict['listingCountsByState'])
df_featuredContent = pd.DataFrame(my_dict['featuredContent'])
df_homeDesigns = pd.DataFrame(my_dict['homeDesigns'])
df_newlyReleased

,id,listingType,agencyName,suburb,state,listingUrl,imageUrl,numberOfBedrooms,numberOfBathrooms,numberOfCarspaces
0,2018578535,Residential,Redink Homes Southwest,West Busselton,WA,https://www.domain.com.au/seymour-street-west-...,https://rimh2.domainstatic.com.au/jV2AClxkGvKg...,3,2,2.0
1,2018578534,Residential,Redink Homes Southwest,Dalyellup,WA,https://www.domain.com.au/topaz-way-dalyellup-...,https://rimh2.domainstatic.com.au/42qqAH0bC97A...,4,2,2.0
2,2018578533,Residential,Redink Homes Southwest,Glen Iris,WA,https://www.domain.com.au/barker-boulevard-gle...,https://rimh2.domainstatic.com.au/Po9l7FO2WouS...,4,2,2.0
3,2018578532,Residential,Redink Homes Southwest,Eaton,WA,https://www.domain.com.au/blue-wren-drive-eato...,https://rimh2.domainstatic.com.au/8I0IMl-ZaTGa...,4,2,2.0
4,2018578531,Residential,Redink Homes Southwest,Australind,WA,https://www.domain.com.au/iolite-lane-australi...,https://rimh2.domainstatic.com.au/L93OpI0bCPjG...,4,2,2.0
5,2018578529,Residential,Redink Homes Southwest,Australind,WA,https://www.domain.com.au/centaurus-avenue-aus...,https://rimh2.domainstatic.com.au/FRTbWXPP0v8D...,4,2,2.0
6,2018575980,Residential,B1 Homes,Riverton,WA,https://www.domain.com.au/riverton-wa-6148-201...,https://rimh2.domainstatic.com.au/eYsdoO3TKX1T...,3,2,1.0
7,2018578516,Residential,B1 Homes,Southern River,WA,https://www.domain.com.au/lot-337-composure-wa...,https://rimh2.domainstatic.com.au/h1E5AOpM5eIo...,2,2,NaN
8,2018578489,Residential,B1 Homes,Whitby,WA,https://www.domain.com.au/lot-491-heartwood-bl...,https://rimh2.domainstatic.com.au/N7ZCEy28cChb...,4,2,2.0
9,2018578476,Residential,B1 Homes,Wellard,WA,https://www.domain.com.au/lot-860-framingham-c...,https://rimh2.domainstatic.com.au/sU5PSQv-ldwf...,4,2,2.0


In [ ]:
df_listingCountsByState

,state,totalResults,imageUrl
0,NSW,3136,https://rimh2.domainstatic.com.au/uickwwXbsxB9...
1,VIC,2912,https://rimh2.domainstatic.com.au/i5uwaFtneUgE...
2,QLD,3810,https://rimh2.domainstatic.com.au/jkymH6iJesk-...
3,ACT,26,https://rimh2.domainstatic.com.au/-hYy4JuOwO8B...
4,NT,7,https://rimh2.domainstatic.com.au/WTDrXcAw4Up1...
5,SA,523,https://rimh2.domainstatic.com.au/DApvy11YacSa...
6,WA,3640,https://rimh2.domainstatic.com.au/eQvBGZtcd-dn...
7,TAS,42,https://rimh2.domainstatic.com.au/HqihYNSjjEWt...


In [ ]:
df_featuredContent

,ArticleID,ArticleCat,ArticleUrl,ImageUrl,Title
0,928792,Building,https://www.domain.com.au/advice/the-ultimate-...,https://res.akamaized.net/domain/image/upload/...,The ultimate guide to building a new home
1,921009,Building,https://www.domain.com.au/advice/what-to-look-...,https://res.akamaized.net/domain/image/upload/...,What to look for when buying into a new suburb
2,917953,Building,https://www.domain.com.au/advice/budget-mid-ra...,https://res.akamaized.net/domain/image/upload/...,How much does it really cost to install a swim...
3,849950,Building,https://www.domain.com.au/advice/five-common-c...,https://res.akamaized.net/domain/image/upload/...,Five common causes for budget blow out when bu...


In [ ]:
df_homeDesigns

,id,listingType,agencyName,suburb,state,listingUrl,imageUrl,numberOfBedrooms,numberOfBathrooms,numberOfCarspaces,projectName
0,2018576948,Residential,Perth Home Brokers,Brabham,WA,https://www.domain.com.au/brabham-wa-6055-2018...,https://rimh2.domainstatic.com.au/ePu6WKAyM97V...,4.0,2.0,2.0,NaN
1,2018572039,Residential,Property1group,Oakville,NSW,https://www.domain.com.au/oakville-nsw-2765-20...,https://rimh2.domainstatic.com.au/9UKUCcNA1JR3...,3.0,2.0,1.0,NaN
2,5953,Project,McDonald Jones Homes,Leppington,NSW,https://www.domain.com.au/project/5953/mjh-in-...,https://rimh2.domainstatic.com.au/z6qodPeKqs7g...,NaN,NaN,NaN,MJH in Leppington
3,2018463239,Residential,@realty,Vacy,NSW,https://www.domain.com.au/22-clarkes-crossing-...,https://rimh2.domainstatic.com.au/-EhWc7GYEcM3...,4.0,3.0,2.0,NaN
4,5954,Project,McDonald Jones Homes,Thornton,NSW,https://www.domain.com.au/project/5954/mjh-in-...,https://rimh2.domainstatic.com.au/dkiL3xbKc_Zz...,NaN,NaN,NaN,MJH in Thornton
5,5952,Project,McDonald Jones Homes,Box Hill,NSW,https://www.domain.com.au/project/5952/mjh-in-...,https://rimh2.domainstatic.com.au/vVKTMYfSvtCU...,NaN,NaN,NaN,MJH in Box Hill
6,5647,Project,Colliers Newcastle | Sovereign Park,Newcastle,NSW,https://www.domain.com.au/project/5647/soverei...,https://rimh2.domainstatic.com.au/wp1FO-Cypdrh...,NaN,NaN,NaN,Sovereign Park
7,2690,Project,Frasers Property NSW,Shell Cove,NSW,https://www.domain.com.au/project/2690/the-wat...,https://rimh2.domainstatic.com.au/2jRGR35sS31w...,NaN,NaN,NaN,The Waterfront
8,4748,Project,Mulpha Sanctuary Cove Developments Pty Ltd,Sanctuary Cove,QLD,https://www.domain.com.au/project/4748/horizon...,https://rimh2.domainstatic.com.au/00YwF3-Lbw1w...,NaN,NaN,NaN,Horizon
9,2018568234,Residential,Macquarie Home Group Pty Ltd,Farley,NSW,https://www.domain.com.au/lot-236-farley-rise-...,https://rimh2.domainstatic.com.au/krvGRrLj1Rqn...,4.0,2.0,2.0,NaN


In [ ]:
!pip install selenium
!apt-get update
!apt-get install firefox -y
!pip install selenium

# Download and install GeckoDriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.30.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InReleas

In [ ]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from selenium import webdriver

In [ ]:
# Set up options
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Set up driver
driver = webdriver.Firefox(options=options)

In [ ]:
url = 'https://www.domain.com.au/'
driver.get(url)

# Get the page source and parse it with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [ ]:

dream_home_items = soup.find_all('div', class_='css-1xvlhm5')

# Iterate over each dream home item
for item in dream_home_items:
    # Extract the data from each item
    try:
      link = item.find('a', class_='css-89y0yw')['href']
    except:
      None
    try:
      image = item.find('img', class_='css-1oisjlp')['src']
    except:
      None
    try:    
      subtitle = item.find('span', class_='dream-homes__subtitle').text
    except:
      None
    try:
      location = item.find('span', class_='css-19z2o7j').text
    except:
      None
    try:
      state = item.find('span', class_='css-1l6h6nn').text
    except:
      None
    try:
      bedrooms = item.find('li', class_='dream-homes__feature-bedroom').text.strip()
    except:
      None
    try:
      bathrooms = item.find('li', class_='dream-homes__feature-bathroom').text.strip()
    except:
      None
    try:
      carspaces = item.find('li', class_='dream-homes__feature-carspace').text.strip()
    except:
      None

    # Print the extracted data
    print('Link:', link)
    print('Image:', image)
    print('Subtitle:', subtitle)
    print('Location:', location)
    print('State:', state)
    print('Bedrooms:', bedrooms)
    print('Bathrooms:', bathrooms)
    print('Carspaces:', carspaces)
    print('---')

Link: https://www.domain.com.au/52a-avanti-street-mermaid-waters-qld-4218-2018545745
Image: https://strap.domain.com.au/dream-homes-nsw/DreamHomes2018545745.jpg
Subtitle: London Estate Agents
Location: Mermaid Waters
State: Qld
Bedrooms: 4bedrooms
Bathrooms: 3bathrooms
Carspaces: 2carspaces
---
Link: https://www.domain.com.au/project/5469/rumbalara-residences-gosford-nsw
Image: https://strap.domain.com.au/dream-homes-nsw/DreamHomes5469.jpg
Subtitle: Rumbalara Residences
Location: Gosford
State: NSW
Bedrooms: 4bedrooms
Bathrooms: 3bathrooms
Carspaces: 2carspaces
---
Link: https://www.domain.com.au/project/3561/the-landmark-st-leonards-nsw
Image: https://strap.domain.com.au/dream-homes-nsw/DreamHomes3561.jpg
Subtitle: The  Landmark
Location: St Leonards
State: NSW
Bedrooms: 4bedrooms
Bathrooms: 3bathrooms
Carspaces: 2carspaces
---
